<a href="https://colab.research.google.com/github/jhuarancca/DSRP-DataArchitect/blob/main/S7_Validaci%C3%B3n_de_integridad_de_datos_para_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S7 - Validación de integridad de datos para ML

<img src="https://www.irion-edm.com/wp-content/uploads/2021/01/DataQuality-che-cose-perche-adottarla-e-come-applicarla.png" width="590" height="300">

Docente: Miguel Yepez

---

# Caso 1

In [ ]:
!pip install pandera

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pandera as pa
from pandera import Column, DataFrameSchema, Check
from IPython.display import display  # para mostrar la tabla de errores

In [ ]:
# 🔹 Dataset simulado con errores de calidad intencionales
df = pd.DataFrame({
    "cliente_id": [101, 102, 102, None, 105, 106, 107],
    "fecha": ["2023-01-01", "2023-01-01", "2023-01-02", "2023-01-03", "malafecha", "2023-01-05", "2023-01-06"],
    "monto": [100.0, 150.0, None, -30.0, 200.0, 80.0, 60.0],
    "estado": ["ACTIVO", "ACTIVO", "INACTIVO", "ACTIVO", "PENDIENTE", "INACTIVO", "ACTIVO"],
    "devolucion": [0.0, -10.0, 0.0, 20.0, -5.0, None, -999.0],
    "correo": [
        "cliente1@empresa.com",  # válido
        "cliente2@empresa.com",  # válido
        "cliente2@empresa.com",  # duplicado
        "cliente4empresa.com",   # sin @
        "cliente5@empresa",      # sin dominio
        None,                    # nulo
        "cliente7@empresa.com"   # válido
    ]
})

In [ ]:
# 🔹 Conversión inicial de fechas (para evitar errores de tipo)
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

In [ ]:
df

,cliente_id,fecha,monto,estado,devolucion,correo
0,101.0,2023-01-01,100.0,ACTIVO,0.0,cliente1@empresa.com
1,102.0,2023-01-01,150.0,ACTIVO,-10.0,cliente2@empresa.com
2,102.0,2023-01-02,NaN,INACTIVO,0.0,cliente2@empresa.com
3,NaN,2023-01-03,-30.0,ACTIVO,20.0,cliente4empresa.com
4,105.0,NaT,200.0,PENDIENTE,-5.0,cliente5@empresa
5,106.0,2023-01-05,80.0,INACTIVO,NaN,None
6,107.0,2023-01-06,60.0,ACTIVO,-999.0,cliente7@empresa.com


In [ ]:
# 🔹 Definición del esquema con reglas de calidad alineadas a DAMA
schema = DataFrameSchema({
    "cliente_id": Column(pa.Int, nullable=False, unique=True),
    "fecha": Column(pa.DateTime, nullable=False),
    "monto": Column(pa.Float, checks=[Check.ge(0)], nullable=False),
    "estado": Column(pa.String, checks=Check.isin(["ACTIVO", "INACTIVO"])),
    "devolucion": Column(pa.Float, checks=[Check.le(0)]),
    "correo": Column(
        pa.String,
        nullable=False,
        checks=[
            Check.str_matches(r"^[\w\.-]+@[\w\.-]+\.\w{2,4}$")  # Validación de formato correo
        ],
        unique=True
    )
})

schema

<Schema DataFrameSchema(columns={'cliente_id': <Schema Column(name=cliente_id, type=DataType(int64))>, 'fecha': <Schema Column(name=fecha, type=DataType(datetime64[ns]))>, 'monto': <Schema Column(name=monto, type=DataType(float64))>, 'estado': <Schema Column(name=estado, type=DataType(str))>, 'devolucion': <Schema Column(name=devolucion, type=DataType(float64))>, 'correo': <Schema Column(name=correo, type=DataType(str))>}, checks=[], parsers=[], index=None, dtype=None, coerce=False, strict=False, name=None, ordered=False, unique=None, report_duplicates=all, unique_column_names=False, add_missing_columns=False, title=None, description=None, metadata=None, drop_invalid_rows=False)>

In [ ]:
# 🔹 Validación con visualización de errores
try:
    schema.validate(df, lazy=True)
except pa.errors.SchemaErrors as err:
    errores_df = err.failure_cases.copy()
    errores_df.columns = ["Contexto", "Columna", "Regla fallida", "Número de chequeo", "Valor con error", "Índice"]
    errores_df = errores_df[["Índice", "Columna", "Regla fallida", "Valor con error"]]
    print("🚨 Errores de validación detectados:")
    display(errores_df)


🚨 Errores de validación detectados:


,Índice,Columna,Regla fallida,Valor con error
0,3,cliente_id,not_nullable,NaN
1,1,cliente_id,field_uniqueness,102.0
2,2,cliente_id,field_uniqueness,102.0
3,4,fecha,not_nullable,NaN
4,2,monto,not_nullable,NaN
5,3,monto,greater_than_or_equal_to(0),-30.0
6,4,estado,"isin(['ACTIVO', 'INACTIVO'])",PENDIENTE
7,5,devolucion,not_nullable,NaN
8,3,devolucion,less_than_or_equal_to(0),20.0
9,5,correo,not_nullable,None


# Caso 2

In [ ]:
# 🔹 Corrección del dataframe para que pase la validación
df_limpio = pd.DataFrame({
    "cliente_id": [101, 102, 103, 104, 105, 106, 107],
    "fecha": pd.to_datetime([
        "2023-01-01", "2023-01-01", "2023-01-02",
        "2023-01-03", "2023-01-04", "2023-01-05", "2023-01-06"
    ]),
    "monto": [100.0, 150.0, 120.0, 130.0, 200.0, 80.0, 60.0],
    "estado": ["ACTIVO", "ACTIVO", "INACTIVO", "ACTIVO", "ACTIVO", "INACTIVO", "ACTIVO"],
    "devolucion": [-0.0, -10.0, -5.0, -2.0, -5.0, -0.1, -0.0],
    "correo": [
        "cliente1@empresa.com",
        "cliente2@empresa.com",
        "cliente3@empresa.com",
        "cliente4@empresa.com",
        "cliente5@empresa.com",
        "cliente6@empresa.com",
        "cliente7@empresa.com"
    ]
})

# 🔹 Validación exitosa
schema.validate(df_limpio)
print("✅ Validación completada sin errores.")


✅ Validación completada sin errores.


---

FIN

---